In [2]:
#preprecess imports

import pandas as pd
import spacy
import numpy as np
import en_core_web_sm
# nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()
from tqdm.notebook import tqdm 

# Essentials
import base64
import re
from tqdm import tqdm
import numpy as np
import pandas as pd

# Gensim and LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this 

# NLP stuff
import contractions
import demoji
import string
import nltk
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#nltk.download('wordnet')

# Plotting tools
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
%matplotlib inline

#sklearn 


In [3]:
df = pd.read_csv('practice.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df2 = pd.read_csv('practice2.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df3 = pd.read_csv('practice3.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df4 = pd.read_csv('practice4.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df5 = pd.read_csv('practice5.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])

li = []

li.append(df)
li.append(df2)
li.append(df3)
li.append(df4)
li.append(df5)
    
frame = pd.concat(li, axis = 0, ignore_index = True)

df = frame

In [4]:
df = df.dropna(axis='rows')

In [5]:
df.head()

,position_section,hl1,hl2,lede,body
1,SPORTS,Wilfork goes out a legend,Lineman leaves Pats for Texans,Vince Wilfork will be immortalized as a Patrio...,'Few players reached or will ever reach the sp...
2,OPINION,Hillary's modus operandi not pretty,'Nasty' GOP latest excuse for private email use,Just when you thought we were done with emailg...,Didn't Clinton tell us that she set up the pri...
6,BUSINESS,RETAILERS FEAR TRASHED STREETS,Mess left beneath snow piles could slow return...,With dirty snowbanks still lining many streets...,'After this very tough winter for main streets...
7,FEATURES,Night of the laughing dead,"CW's 'iZombie' mixes horror, mystery, comedy",It's the 'Walking Dead' 'Quincy'/'Psych' mash...,"One night, the super-overachiever decides to t..."
8,SPORTS,Sox see what's missing,Harvey shows off top-of-rotation arm,"FORT MYERS - It's not just the results, the co...","The Red Sox don't have a Matt Harvey, and they..."


### Importing with Dask

In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd

In [ ]:
df = dd.read_csv("/projectnb/sparkgrp/ds-naacp-media-bias/TBG_unique_raw.csv", usecols = ['position_section', 'hl1', 'hl2', 'lede', 'body'])
df = df.map_partitions(lambda x: x.fillna(''))
df = df.map_partitions(lambda x: x[['hl1', 'hl2', 'lede', 'body']].agg(' '.join, axis = 1))

In [ ]:
df.partitions[5].compute()

### Preprocessing with Badnani's method

In [8]:
#/projectnb/sparkgrp/ds-naacp-media-bias
# def combine_led_body(df):
#     body = df['body'].values
#     lede = df['lede'].values
#     arr = np.vstack((lede,body)).T
#     for step, x in enumerate(arr): 
#         arr[step] = ' '.join(x)

#     arr = arr.flatten()
#     return arr 


df = pd.read_csv('practice.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df2 = pd.read_csv('practice2.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df3 = pd.read_csv('practice3.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df4 = pd.read_csv('practice4.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])
df5 = pd.read_csv('practice5.csv', usecols = ['position_section','hl1', 'hl2', 'lede', 'body'])

li = []

li.append(df)
li.append(df2)
li.append(df3)
li.append(df4)
li.append(df5)
    
frame = pd.concat(li)

df = frame
df.fillna('', inplace = True)
df['content'] = df[['hl1', 'hl2', 'lede', 'body']].agg(' '.join, axis=1)
df.drop(columns = ['hl1', 'hl2', 'lede', 'body'], inplace = True)

def custom_preprocess(corpus_iterable):
    return [['<s>'] + [z.lemma_.lower() for z in y if not (z.is_stop or z.text == '\n')] + ['</s>']
            for x in tqdm(nlp.pipe(corpus_iterable), total = len(corpus_iterable))
            for y in x.sents]

with nlp.select_pipes(disable=['tok2vec', 'parser', 'ner']):
    nlp.add_pipe("sentencizer")
    corpus = []
    temp = []

    for x in tqdm(nlp.pipe(df.content.values), total = len(df.content.values)):
        for y in x.sents:
            temp.append('<s>')
            for z in y:
                if not (z.is_stop or z.text == '\n'):
                    temp.append(z)
            temp.append('</s>')
        corpus.append(temp)
        temp = []
    #corpus = custom_preprocess(df.content.values)




100%|████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:04<00:00, 52.16it/s]


In [ ]:
#corpus = combine_led_body(df)
df.content.values

In [27]:
df_temp = df.content

In [28]:
df_temp.to_csv('preprocessed.csv')

In [11]:
len(corpus) #each body+lede combination is represented twice in corpus

256

In [ ]:
corpus

In [ ]:
def custom_preprocess(corpus_iterable): 
    return [[z.lemma_.lower() for z in y if not (z.is_stop or z.text == '\n')] for y in 
tqdm(nlp.pipe(corpus_iterable), total = len(corpus_iterable)) ]

In [ ]:
pre_processed_corpus = custom_preprocess(corpus)

## TDS LDA Preprocessing

In [ ]:
def preprocess(text_col):
    """This function will apply NLP preprocessing lambda functions over a pandas series such as df['text'].
       These functions include converting text to lowercase, removing emojis, expanding contractions, removing punctuation,
       removing numbers, removing stopwords, lemmatization, etc."""
    
    # convert to lowercase
    text_col = text_col.apply(lambda x: ' '.join([w.lower() for w in x.split()]))
    
    # remove emojis
    text_col = text_col.apply(lambda x: demoji.replace(x, ""))
    
    # expand contractions  
    text_col = text_col.apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))

    # remove punctuation
    text_col = text_col.apply(lambda x: ''.join([i for i in x if i not in string.punctuation]))
    
    # remove numbers
    text_col = text_col.apply(lambda x: ' '.join(re.sub("[^a-zA-Z]+", " ", x).split()))

    # remove stopwords
    stopwords = [sw for sw in nltk.corpus.stopwords.words('english') if sw not in ['not', 'no']]
    text_col = text_col.apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))

    # lemmatization
    text_col = text_col.apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(w) for w in x.split()]))

    # remove short words
    text_col = text_col.apply(lambda x: ' '.join([w.strip() for w in x.split() if len(w.strip()) >= 3]))

    return text_col

In [ ]:
print(preprocess(df['body']))
preprocessed_df = preprocess(df['body'])

In [ ]:
print(preprocessed_df.values[0])

In [ ]:
print(df['body'].values[0])

In [ ]:
df

## Working LDA Topic Modeling

In [9]:
# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
from sklearn.feature_extraction.text import CountVectorizer

documents = pd.read_csv('practice.csv', error_bad_lines=False)
documents = documents.dropna(axis='rows')
documents = df
documents.head()


vect = CountVectorizer(stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(documents.content)
# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`
ldamodel = gensim.models.LdaMulticore(corpus=corpus, id2word=id_map, passes=30,
                                               random_state=42, num_topics=20, workers=2, per_word_topics = True)

/var/folders/gd/pxcx2w0x59j2bsjbzhy62nvw0000gn/T/ipykernel_27028/376646926.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  documents = pd.read_csv('practice.csv', error_bad_lines=False)


In [10]:
for idx, topic in ldamodel.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.015*"said" + 0.005*"trump" + 0.005*"mullin" + 0.004*"officer" + 0.004*"navy" + 0.004*"school" + 0.004*"gallagher" + 0.004*"president" + 0.004*"companies" + 0.004*"saunders"


Topic: 1 
Words: 0.015*"said" + 0.005*"new" + 0.005*"clark" + 0.005*"students" + 0.004*"weinstein" + 0.003*"people" + 0.003*"week" + 0.003*"like" + 0.003*"acting" + 0.003*"woman"


Topic: 2 
Words: 0.011*"trump" + 0.010*"said" + 0.005*"trade" + 0.005*"new" + 0.005*"state" + 0.004*"deal" + 0.004*"image" + 0.003*"link" + 0.003*"000" + 0.003*"walsh"


Topic: 3 
Words: 0.015*"said" + 0.007*"artists" + 0.006*"day" + 0.005*"booster" + 0.005*"new" + 0.004*"dose" + 0.004*"space" + 0.004*"building" + 0.004*"moderna" + 0.004*"office"


Topic: 4 
Words: 0.018*"said" + 0.005*"long" + 0.004*"boston" + 0.004*"like" + 0.003*"time" + 0.003*"year" + 0.003*"people" + 0.003*"government" + 0.003*"new" + 0.003*"saturday"


Topic: 5 
Words: 0.011*"said" + 0.010*"state" + 0.005*"states" + 0.005*"baker" + 0.005*"family

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

In [ ]:
' '.join(df.body.tolist() ).lower() 

In [ ]:
#filter the df to one candidate, and create a list of responses from them
import warnings
warnings.filterwarnings('ignore')

for i in df.body.values:
    text = i

    # join the list and lowercase all the words
    #text = ' '.join([text]).lower()

    #create the wordcloud object
    wordcloud = WordCloud(stopwords = STOPWORDS,
                          collocations=True).generate(text)

    #plot the wordcloud object
    plt.imshow(wordcloud, interpolation='bilInear')
    plt.axis('off')
    plt.show()

In [ ]:

def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    return output
 
topic_distribution(documents.body[1])

## Second Approach to topic modeling using PAM

In [ ]:
import tomotopy as tp
mdl = tp.PAModel(k1=5, k2=25)
for docs in df.content.values:
    mdl.add_doc(docs)
mdl.train(1000)

# first, infer the document
doc_inst = mdl.make_doc(df.content.values[0])
mdl.infer(doc_inst)

# next, count subtopics
subtopic_counts = [0] * mdl.k2
for t in doc_inst.subtopics:
    subtopic_counts[t] += 1

# estimate distribution
total = sum(subtopic_counts)
for i in range(mdl.k2):
    print('Subtopic #{} : {}'.format(i, subtopic_counts[i] / total))


In [ ]:
print(df.content.values[0])

In [1]:
# currently struggling to make sense of subtopic outputs

## Automatic labeling with Tomotopy

In [14]:
import tomotopy as tp

In [29]:
corpus = tp.utils.Corpus(tokenizer=tp.utils.SimpleTokenizer(), stopwords=['.'])
# data_feeder yields a tuple of (raw string, user data) or a str (raw string)
corpus.process(open("preprocessed.csv", encoding='utf-8'))

257

In [33]:
# make LDA model and train
mdl = tp.LDAModel(k=10, min_cf=10, min_df=5, corpus=corpus)
mdl.train(0)
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
print('Removed top words:', mdl.removed_top_words)
for i in range(0, 1000, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

Num docs: 257 , Vocab size: 1961 , Num words: 129479
Removed top words: []
Iteration: 0	Log-likelihood: -7.3253967857652516
Iteration: 10	Log-likelihood: -7.07386861610103
Iteration: 20	Log-likelihood: -6.967255645934394
Iteration: 30	Log-likelihood: -6.91065479245327
Iteration: 40	Log-likelihood: -6.869271837660496
Iteration: 50	Log-likelihood: -6.83428178643925
Iteration: 60	Log-likelihood: -6.818519522383362
Iteration: 70	Log-likelihood: -6.797362179659312
Iteration: 80	Log-likelihood: -6.780528986881967
Iteration: 90	Log-likelihood: -6.755442248524666
Iteration: 100	Log-likelihood: -6.74060331267081
Iteration: 110	Log-likelihood: -6.736824803320526
Iteration: 120	Log-likelihood: -6.728048811210432
Iteration: 130	Log-likelihood: -6.7105131141227945
Iteration: 140	Log-likelihood: -6.703054249220132
Iteration: 150	Log-likelihood: -6.697962539381938
Iteration: 160	Log-likelihood: -6.679774014627596
Iteration: 170	Log-likelihood: -6.664726606834184
Iteration: 180	Log-likelihood: -6.6732

In [34]:
# extract candidates for auto topic labeling
extractor = tp.label.PMIExtractor(min_cf=10, min_df=5, max_len=5, max_cand=10000)
cands = extractor.extract(mdl)

In [35]:
# ranking the candidates of labels for a specific topic
labeler = tp.label.FoRelevance(mdl, cands, min_df=5, smoothing=1e-2, mu=0.25)
for k in range(mdl.k):
    print("== Topic #{} ==".format(k))
    print("Labels:", ', '.join(label for label, score in labeler.get_topic_labels(k, top_n=5)))
    for word, prob in mdl.get_topic_words(k, top_n=10):
        print(word, prob, sep='\t')
    print()

== Topic #0 ==
Labels: 2016 election, military aid, the Intelligence, the President and, Biden and
the	0.08113724738359451
to	0.05791759490966797
president	0.049000538885593414
trump	0.03240245580673218
s	0.02639889530837536
that	0.026134032756090164
of	0.025692595168948174
and	0.01862958073616028
his	0.015980949625372887
house	0.014038621447980404

== Topic #1 ==
Labels: last season, a game, this season, starters, game and
the	0.06602651625871658
to	0.03563389554619789
s	0.032722726464271545
for	0.02573591284453869
team	0.02131093293428421
game	0.018981996923685074
season	0.016653060913085938
games	0.015837932005524635
we	0.015721485018730164
a	0.015022804960608482

== Topic #2 ==
Labels: advocates, groups, laws, Legislature, Democrat
law	0.031009608879685402
school	0.028754569590091705
students	0.027908930554986
that	0.02621765062212944
state	0.02283509261906147
said	0.018888773396611214
for	0.018888773396611214
laws	0.01606997288763523
university	0.014942453242838383
an	0.0132511742